In [2]:
import os
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from efficientnet_pytorch import EfficientNet
from PIL import Image
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# Constants
BATCH = 16
IM_SIZE = 224
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load the data
df = pd.read_csv("train.csv")

# Filter dataset with hotel_ids >= 20
s1 = df["hotel_id"].value_counts()
s2 = s1[s1 >= 20]
hotel_id_list = s2.index.tolist()
df1 = df[df["hotel_id"].isin(hotel_id_list)]

# Train-test split (90:10)
train_df, test_df = train_test_split(df1, test_size=0.1, stratify=df1['hotel_id'], random_state=42)

# Map hotel_id to indices for classification
unique_hotel_ids = sorted(train_df['hotel_id'].unique())
hotel_id_to_index = {hotel_id: idx for idx, hotel_id in enumerate(unique_hotel_ids)}
index_to_hotel_id = {idx: hotel_id for hotel_id, idx in hotel_id_to_index.items()}

# Add the class index to the DataFrame
train_df['class_index'] = train_df['hotel_id'].map(hotel_id_to_index)
test_df['class_index'] = test_df['hotel_id'].map(hotel_id_to_index)

# Define transformations
Transform = transforms.Compose([
    transforms.Resize((IM_SIZE, IM_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

# Dataset Class
class HotelImageDataset(Dataset):
    def __init__(self, df, image_dir, transform=None):
        self.df = df
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_name = self.df.iloc[idx]['image']
        label = self.df.iloc[idx]['class_index']
        img_path = os.path.join(self.image_dir, str(self.df.iloc[idx]['chain']), img_name)
        
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        
        return image, label

# Initialize datasets and loaders
image_dir = r"C:\Users\agnit\Documents\UT Austin\Fall'24\Adv Machine Learning\Project\hotel-id-2021-fgvc8\train_images"
train_dataset = HotelImageDataset(train_df, image_dir, Transform)
test_dataset = HotelImageDataset(test_df, image_dir, Transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH, shuffle=False)

# Load EfficientNet-B7 and remove the final classification layer
model = EfficientNet.from_pretrained('efficientnet-b7')
model._fc = torch.nn.Identity()  # Replace the final layer with an identity layer
model = model.to(DEVICE)

# Extract features
def extract_features(loader):
    model.eval()
    features = []
    labels = []
    with torch.no_grad():
        for images, lbls in tqdm(loader, desc="Extracting Features", unit="batch"):
            images = images.to(DEVICE)
            feats = model(images)  # Get features from EfficientNet
            features.append(feats.cpu())
            labels.extend(lbls.numpy())
    return torch.cat(features).numpy(), labels

train_features, train_labels = extract_features(train_loader)
test_features, test_labels = extract_features(test_loader)

# Train logistic regression
logreg = LogisticRegression(max_iter=1000)
logreg.fit(train_features, train_labels)

# Predict on test features
test_preds = logreg.predict(test_features)

# Evaluate
accuracy = accuracy_score(test_labels, test_preds)
print(f"Test Accuracy: {accuracy:.4f}")

# Save predictions
test_df['predicted_class_index'] = test_preds
test_df['predicted_hotel_id'] = test_df['predicted_class_index'].map(index_to_hotel_id)
test_df.to_csv("test_predictions.csv", index=False)


Loaded pretrained weights for efficientnet-b7


Extracting Features: 100%|████████████████████████████████████████████████████████| 240/240 [12:55<00:00,  3.23s/batch]


Test Accuracy: 0.3931


PermissionError: [Errno 13] Permission denied: 'test_predictions.csv'